# Deep Q-learning (DQL) or Deep Q-network (DQN)

In this notebook, we'll build a neural network that can learn to play games through reinforcement learning. More specifically, we'll use Q-learning to train an agent to play a game called [Cart-Pole](https://gym.openai.com/envs/CartPole-v0). In this game, a freely swinging pole is attached to a cart. The cart can move to the left and right, and the goal is to keep the pole upright as long as possible.

![Cart-Pole](assets/cart-pole.jpg)

We can simulate this game using [OpenAI Gym](https://gym.openai.com/). First, let's check out how OpenAI Gym works. Then, we'll get into training an agent to play the Cart-Pole game.

In [1]:
import numpy as np

In [2]:
import tensorflow as tf
print('TensorFlow Version: {}'.format(tf.__version__))
print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

TensorFlow Version: 1.12.0
Default GPU Device: /device:GPU:0


>**Note:** Make sure you have OpenAI Gym cloned into the same directory with this notebook. I've included `gym` as a submodule, so you can run `git submodule --init --recursive` to pull the contents into the `gym` repo.

>**Note:** Make sure you have OpenAI Gym cloned. Then run this command `pip install -e gym/[all]`.

In [3]:
import gym

## Create the Cart-Pole game environment
# env = gym.make('CartPole-v0') # 200 total reward as goal
env = gym.make('CartPole-v1') # 500 total reward as goal

We interact with the simulation through `env`. To show the simulation running, you can use `env.render()` to render one frame. Passing in an action as an integer to `env.step` will generate the next step in the simulation.  You can see how many actions are possible from `env.action_space` and to get a random action you can use `env.action_space.sample()`. This is general to all Gym games. In the Cart-Pole game, there are two possible actions, moving the cart left or right. So there are two actions we can take, encoded as 0 and 1.

Run the code below to watch the simulation run.

In [4]:
# env.reset()
# batch = []
# for _ in range(1000):
#     # env.render()
#     action = env.action_space.sample()
#     state, reward, done, info = env.step(action) # take a random action
#     batch.append([action, state, reward, done, info])
#     #print('state, action, reward, done, info:', state, action, reward, done, info)
#     if done:
#         env.reset()

To shut the window showing the simulation, use `env.close()`.

If you ran the simulation above, we can look at the rewards:

In [5]:
# batch[0], batch[0][1].shape

In [6]:
# actions = np.array([each[0] for each in batch])
# states = np.array([each[1] for each in batch])
# rewards = np.array([each[2] for each in batch])
# dones = np.array([each[3] for each in batch])
# infos = np.array([each[4] for each in batch])

In [7]:
# print(rewards[-20:])
# print(np.array(rewards).shape, np.array(states).shape, np.array(actions).shape, np.array(dones).shape)
# print(np.array(rewards).dtype, np.array(states).dtype, np.array(actions).dtype, np.array(dones).dtype)
# print(np.max(np.array(actions)), np.min(np.array(actions)))
# print((np.max(np.array(actions)) - np.min(np.array(actions)))+1)
# print(np.max(np.array(rewards)), np.min(np.array(rewards)))
# print(np.max(np.array(states)), np.min(np.array(states)))

The game resets after the pole has fallen past a certain angle. For each frame while the simulation is running, it returns a reward of 1.0. The longer the game runs, the more reward we get. Then, our network's goal is to maximize the reward by keeping the pole vertical. It will do this by moving the cart to the left and the right.

## Q-Network

We train our Q-learning agent using the Bellman Equation:

$$
Q(s, a) = r + \gamma \max{Q(s', a')}
$$

where $s$ is a state, $a$ is an action, and $s'$ is the next state from state $s$ and action $a$.

Before we used this equation to learn values for a Q-_table_. However, for this game there are a huge number of states available. The state has four values: the position and velocity of the cart, and the position and velocity of the pole. These are all real-valued numbers, so ignoring floating point precisions, you practically have infinite states. Instead of using a table then, we'll replace it with a neural network that will approximate the Q-table lookup function.

<img src="assets/deep-q-learning.png" width=450px>

Now, our Q value, $Q(s, a)$ is calculated by passing in a state to the network. The output will be Q-values for each available action, with fully connected hidden layers.

<img src="assets/q-network.png" width=550px>


As I showed before, we can define our targets for training as $\hat{Q}(s,a) = r + \gamma \max{Q(s', a')}$. Then we update the weights by minimizing $(\hat{Q}(s,a) - Q(s,a))^2$. 

For this Cart-Pole game, we have four inputs, one for each value in the state, and two outputs, one for each action. To get $\hat{Q}$, we'll first choose an action, then simulate the game using that action. This will get us the next state, $s'$, and the reward. With that, we can calculate $\hat{Q}$ then pass it back into the $Q$ network to run the optimizer and update the weights.

Below is my implementation of the Q-network. I used two fully connected layers with ReLU activations. Two seems to be good enough, three might be better. Feel free to try it out.

In [8]:
def model_input(state_size, action_size):
    actions = tf.placeholder(tf.int32, [None], name='actions')
    actions_logits = tf.placeholder(tf.float32, [None, action_size], name='actions_logits')
    states = tf.placeholder(tf.float32, [None, state_size], name='states')
    next_states = tf.placeholder(tf.float32, [None, state_size], name='next_states')
    targetQs = tf.placeholder(tf.float32, [None], name='targetQs')
    is_training = tf.placeholder(tf.bool, name='is_training')
    return actions, actions_logits, states, next_states, targetQs, is_training

In [9]:
def init_xavier(random_seed=1, dtype=tf.float32, uniform=False):
    xavier = tf.contrib.layers.xavier_initializer(
        dtype=dtype,
        seed=tf.set_random_seed(random_seed), 
        uniform=uniform) # False: normal
    return xavier

In [10]:
def mlp(inputs, units, trainable=True):
    outputs = tf.layers.dense(
        inputs=inputs,
        units=units,
        activation=None,
        use_bias=True,
        kernel_initializer=init_xavier(), # Xavier with normal init
        bias_initializer=tf.zeros_initializer(),
        kernel_regularizer=None,
        bias_regularizer=None,
        activity_regularizer=None,
        kernel_constraint=None,
        bias_constraint=None,
        trainable=trainable,
        name=None,
        reuse=None)
    return outputs

In [11]:
# tf.nn.leaky_relu(
#     features,
#     alpha=0.2,
#     name=None
# )
def nl(inputs, alpha=0.2):
    outputs = tf.maximum(alpha * inputs, inputs)
    return outputs

In [12]:
def bn(inputs, training=False):
    outputs = tf.layers.batch_normalization(
        inputs=inputs,
        axis=-1,
        momentum=0.99,
        epsilon=0.001,
        center=True,
        scale=True,
        beta_initializer=tf.zeros_initializer(),
        gamma_initializer=tf.ones_initializer(),
        moving_mean_initializer=tf.zeros_initializer(),
        moving_variance_initializer=tf.ones_initializer(),
        beta_regularizer=None,
        gamma_regularizer=None,
        beta_constraint=None,
        gamma_constraint=None,
        training=training,
        trainable=True,
        name=None,
        reuse=None,
        renorm=False,
        renorm_clipping=None,
        renorm_momentum=0.99,
        fused=None,
        virtual_batch_size=None,
        adjustment=None)
    return outputs

In [14]:
# Critic
def D(states, actions_logits, state_size, hidden_size, reuse=False, alpha=0.2, is_training=False):
    with tf.variable_scope('D', reuse=reuse):
        # First fully connected layer
        h = mlp(inputs=states, units=hidden_size)
        h = bn(inputs=h, training=is_training)
        h = nl(h)
        print(states.shape, h.shape)
        
        # Second fully connected layer
        h = tf.concat([h, actions_logits], axis=1)
        h = mlp(inputs=h, units=hidden_size)
        h = bn(inputs=h, training=is_training)
        h = nl(h)
        print(h.shape)
        
        # Output layer
        Qvalues_ = mlp(inputs=h, units=1)
        return Qvalues_

In [18]:
# Actor
def G(states, action_size, hidden_size, reuse=False, alpha=0.2, is_training=False):
    with tf.variable_scope('G', reuse=reuse):
        # First fully connected layer
        h = mlp(inputs=states, units=hidden_size)
        h = bn(inputs=h, training=is_training)
        h = nl(h)
        print(states.shape, h.shape)
        
        # Second fully connected layer
        h = mlp(inputs=h, units=hidden_size)
        h = bn(inputs=h, training=is_training)
        h = nl(h)
        print(h.shape)
        
        # Output layer
        actions_ = mlp(inputs=h, units=action_size)
        return actions_

In [19]:
def model_loss(actions, actions_logits, states, next_states, targetQs, 
               state_size, action_size, hidden_size, 
               is_training=False):

    Qvalues_ = D(states=states, actions_logits=actions_logits, hidden_size=hidden_size, 
                 state_size=state_size, is_training=is_training)
    
    Qs = tf.reshape(Qvalues_, [-1])
    print('Qvalues_.shape, Qs.shape:', Qvalues_.shape, Qs.shape)
    
    #gloss = tf.reduce_mean(tf.reduce_sum((next_states - next_states_)**2, axis=1))

    actions_logits_ = G(states=states, hidden_size=hidden_size, action_size=action_size, 
                        is_training=is_training)
    
    gQvalues_ = D(states=states, actions_logits=actions_logits_, hidden_size=hidden_size, 
                  state_size=state_size, is_training=is_training, reuse=True)
    
    gQs = tf.reshape(gQvalues_, [-1])
    print('Qvalues_.shape, Qs.shape:', gQvalues_.shape, gQs.shape)
    
    dloss = tf.reduce_mean((Qs - targetQs)**2)
    gloss = -tf.reduce_mean(gQs)
    
    return actions_logits_, gQvalues_, gloss, dloss

In [20]:
def model_opt(gloss, dloss, learning_rate):
    # Get weights and bias to update
    t_vars = tf.trainable_variables()
    g_vars = [var for var in t_vars if var.name.startswith('G')]
    d_vars = [var for var in t_vars if var.name.startswith('D')]

    # Optimize
    with tf.control_dependencies(tf.get_collection(tf.GraphKeys.UPDATE_OPS)): # Required for batchnorm (BN)
        g_opt = tf.train.AdamOptimizer(learning_rate).minimize(gloss, var_list=g_vars)
        d_opt = tf.train.AdamOptimizer(learning_rate).minimize(dloss, var_list=d_vars)

    return g_opt, d_opt

In [21]:
class Model:
    def __init__(self, state_size, action_size, hidden_size, learning_rate):

        # Data of the Model: make the data available inside the framework
        (self.actions, self.actions_logits, self.states, self.next_states, self.targetQs, \
         self.is_training) = model_input(state_size=state_size, action_size=action_size)

        # Create the Model: calculating the loss and forwad pass
        self.actions_logits_, self.gQvalues_, self.gloss, self.dloss = model_loss(
            action_size=action_size, hidden_size=hidden_size, state_size=state_size,
            states=self.states, next_states=self.next_states, 
            actions=self.actions, actions_logits=self.actions_logits,
            targetQs=self.targetQs, is_training=self.is_training)

        # Update the model: backward pass and backprop
        self.g_opt, self.d_opt = model_opt(gloss=self.gloss, dloss=self.dloss, learning_rate=learning_rate)

## Experience replay

Reinforcement learning algorithms can have stability issues due to correlations between states. To reduce correlations when training, we can store the agent's experiences and later draw a random mini-batch of those experiences to train on. 

Here, we'll create a `Memory` object that will store our experiences, our transitions $<s, a, r, s'>$. This memory will have a maxmium capacity, so we can keep newer experiences in memory while getting rid of older experiences. Then, we'll sample a random mini-batch of transitions $<s, a, r, s'>$ and train on those.

Below, I've implemented a `Memory` object. If you're unfamiliar with `deque`, this is a double-ended queue. You can think of it like a tube open on both sides. You can put objects in either side of the tube. But if it's full, adding anything more will push an object out the other side. This is a great data structure to use for the memory buffer.

In [22]:
from collections import deque
class Memory():
    def __init__(self, max_size = 1000):
        self.buffer = deque(maxlen=max_size)
    def sample(self, batch_size):
        idx = np.random.choice(np.arange(len(self.buffer)), 
                               size=batch_size, 
                               replace=False)
        return [self.buffer[ii] for ii in idx]

## Exploration - Exploitation

To learn about the environment and rules of the game, the agent needs to explore by taking random actions. We'll do this by choosing a random action with some probability $\epsilon$ (epsilon).  That is, with some probability $\epsilon$ the agent will make a random action and with probability $1 - \epsilon$, the agent will choose an action from $Q(s,a)$. This is called an **$\epsilon$-greedy policy**.


At first, the agent needs to do a lot of exploring. Later when it has learned more, the agent can favor choosing actions based on what it has learned. This is called _exploitation_. We'll set it up so the agent is more likely to explore early in training, then more likely to exploit later in training.

## Q-Learning training algorithm

Putting all this together, we can list out the algorithm we'll use to train the network. We'll train the network in _episodes_. One *episode* is one simulation of the game. For this game, the goal is to keep the pole upright for 195 frames. So we can start a new episode once meeting that goal. The game ends if the pole tilts over too far, or if the cart moves too far the left or right. When a game ends, we'll start a new episode. Now, to train the agent:

* Initialize the memory $D$
* Initialize the action-value network $Q$ with random weights
* **For** episode = 1, $M$ **do**
  * **For** $t$, $T$ **do**
     * With probability $\epsilon$ select a random action $a_t$, otherwise select $a_t = \mathrm{argmax}_a Q(s,a)$
     * Execute action $a_t$ in simulator and observe reward $r_{t+1}$ and new state $s_{t+1}$
     * Store transition $<s_t, a_t, r_{t+1}, s_{t+1}>$ in memory $D$
     * Sample random mini-batch from $D$: $<s_j, a_j, r_j, s'_j>$
     * Set $\hat{Q}_j = r_j$ if the episode ends at $j+1$, otherwise set $\hat{Q}_j = r_j + \gamma \max_{a'}{Q(s'_j, a')}$
     * Make a gradient descent step with loss $(\hat{Q}_j - Q(s_j, a_j))^2$
  * **endfor**
* **endfor**

## Hyperparameters

One of the more difficult aspects of reinforcememt learning are the large number of hyperparameters. Not only are we tuning the network, but we're tuning the simulation.

In [23]:
# print('state size:{}'.format(states.shape), 
#       'actions:{}'.format(actions.shape)) 
# print('action size:', np.max(actions) - np.min(actions)+1)

In [24]:
# Network parameters
action_size = 2
state_size = 4
hidden_size = 64               # number of units in each Q-network hidden layer
learning_rate = 1e-04         # Q-network learning rate

# Memory parameters
memory_size = int(1e6)            # memory capacity
batch_size = 1024             # experience mini-batch size
gamma = 0.99                 # future reward discount

In [25]:
# Reset/init the graph/session
graph = tf.reset_default_graph()

# Init the model
model = Model(action_size=action_size, hidden_size=hidden_size, state_size=state_size, 
              learning_rate=learning_rate)

# Init the memory
memory = Memory(max_size=memory_size)

(?, 4) (?, 64)
(?, 64)
Qvalues_.shape, Qs.shape: (?, 1) (?,)
(?, 4) (?, 64)
(?, 64)
(?, 4) (?, 64)
(?, 64)
Qvalues_.shape, Qs.shape: (?, 1) (?,)


## Populate the memory (exprience memory)

Here I'm re-initializing the simulation and pre-populating the memory. The agent is taking random actions and storing the transitions in memory. This will help the agent with exploring the game.

In [26]:
# state = env.reset()

# for _ in range(memory_size):
    
#     action = env.action_space.sample()
    
#     next_state, reward, done, _ = env.step(action)
    
#     memory.buffer.append([state, action, next_state, reward, float(done)])
    
#     state = next_state
    
#     if done is True:
#         state = env.reset()

## Training the model

Below we'll train our agent. If you want to watch it train, uncomment the `env.render()` line. This is slow because it's rendering the frames slower than the network can train. But, it's cool to watch the agent get better at the game.

In [27]:
model.gQvalues_

<tf.Tensor 'D_1/dense_2/BiasAdd:0' shape=(?, 1) dtype=float32>

In [28]:
def learn(sess, memory, batch_size):
    
    batch = memory.sample(batch_size)
    states = np.array([each[0] for each in batch])
    actions = np.array([each[1] for each in batch])
    next_states = np.array([each[2] for each in batch])
    rewards = np.array([each[3] for each in batch])
    dones = np.array([each[4] for each in batch])
    actions_logits = np.array([each[5] for each in batch])
    
    nextQvalues_ = sess.run(model.gQvalues_, feed_dict = {model.states: next_states,
                                                          model.is_training: False})
    
    #nextQs = np.max(next_actions_logits_, axis=1) * (1-dones)
    nextQs = nextQvalues_.reshape(-1) * (1-dones)
    
    targetQs = rewards + (gamma * nextQs)
    
    feed_dict = {model.states: states, model.actions: actions, model.actions_logits: actions_logits,
                 model.next_states: next_states, model.targetQs: targetQs, model.is_training: True}
    
    dloss, _ = sess.run([model.dloss, model.d_opt], feed_dict)
    gloss, _ = sess.run([model.gloss, model.g_opt], feed_dict)
                                                             
    return gloss, dloss

In [29]:
# def env_(sess, state, action_logits):
    
#     next_state_ = sess.run(model.next_states_, feed_dict={
#         model.states: state.reshape([1, -1]), 
#         model.actions_logits: action_logits.reshape([1, -1]),
#         model.is_training: False})
    
#     return next_state_.reshape(-1)

In [30]:
def act(sess, state):
    
    action_logits_ = sess.run(model.actions_logits_, feed_dict={model.states: state.reshape([1, -1]), 
                                                                model.is_training: False})
    
    action = np.argmax(action_logits_, axis=1)[0]
    #print(action)
    
    return action, action_logits_.reshape(-1)

In [31]:
# l = []
# l.append([0, 1])

In [32]:
# l[0][0]

In [ ]:
# Save/load the model and save for plotting
saver = tf.train.Saver()
episode_rewards_list, rewards_list, loss_list = [], [], []

# TF session for training
with tf.Session(graph=graph) as sess:
    sess.run(tf.global_variables_initializer())
    #saver.restore(sess, 'checkpoints/model.ckpt')    
    #saver.restore(sess, tf.train.latest_checkpoint('checkpoints'))
    #total_step = 0 # Explore or exploit parameter
    episode_reward = deque(maxlen=100) # 100 episodes average/running average/running mean/window
    
    # Training episodes/epochs
    for ep in range(11111):
        total_reward = 0
        loss_batch = []
        state = env.reset()

        # Training steps/batches
        while True:
            action, action_logits = act(sess, state)
                
            next_state, reward, done, _ = env.step(action)
            
            memory.buffer.append([state, action, next_state, reward, float(done), action_logits])
            
            # # Explore (Env) or Exploit (Model)
            # #total_step += 1
            # #explore_p = explore_stop + (explore_start - explore_stop) * np.exp(-decay_rate * total_step)
            # explore_p = np.random.normal(0, 1)
            # if np.abs(explore_p) < 1e-3:
            #     print('np.abs(explore_p) < 1e-3:', explore_p)
            #     next_state = env_(sess, state=state, action_logits=action_logits)
            
            total_reward += reward
            state = next_state

            # Training
            if len(memory.buffer) >= batch_size:
                gloss, dloss = learn(sess, memory, batch_size)
                loss_batch.append([gloss, dloss])
            
            if done is True:
                break
                
        if len(memory.buffer) >= batch_size:
            episode_reward.append(total_reward)

            print('Episode:{}'.format(ep),
                  'meanR:{:.4f}'.format(np.mean(episode_reward)),
                  'R:{}'.format(total_reward),
                  'gloss:{:.4f}'.format(np.mean(loss_batch, axis=0)[0]),
                  'dloss:{:.4f}'.format(np.mean(loss_batch, axis=0)[1]))
            
            # Ploting out
            episode_rewards_list.append([ep, np.mean(episode_reward)])
            rewards_list.append([ep, total_reward])
            loss_list.append([ep, np.mean(loss_batch)])

            # Break episode/epoch loop
            if np.mean(episode_reward) >= 500:
                break
        
        else: print('len(memory.buffer) >= batch_size:', len(memory.buffer), batch_size)
            
    # At the end of all training episodes/epochs
    saver.save(sess, 'checkpoints/model.ckpt')

len(memory.buffer) >= batch_size: 9 1024
len(memory.buffer) >= batch_size: 18 1024
len(memory.buffer) >= batch_size: 28 1024
len(memory.buffer) >= batch_size: 38 1024
len(memory.buffer) >= batch_size: 48 1024
len(memory.buffer) >= batch_size: 57 1024
len(memory.buffer) >= batch_size: 67 1024
len(memory.buffer) >= batch_size: 77 1024
len(memory.buffer) >= batch_size: 87 1024
len(memory.buffer) >= batch_size: 97 1024
len(memory.buffer) >= batch_size: 106 1024
len(memory.buffer) >= batch_size: 114 1024
len(memory.buffer) >= batch_size: 124 1024
len(memory.buffer) >= batch_size: 134 1024
len(memory.buffer) >= batch_size: 143 1024
len(memory.buffer) >= batch_size: 151 1024
len(memory.buffer) >= batch_size: 165 1024
len(memory.buffer) >= batch_size: 174 1024
len(memory.buffer) >= batch_size: 183 1024
len(memory.buffer) >= batch_size: 191 1024
len(memory.buffer) >= batch_size: 200 1024
len(memory.buffer) >= batch_size: 209 1024
len(memory.buffer) >= batch_size: 217 1024
len(memory.buffer) >= 

Episode:168 meanR:11.5246 R:10.0 gloss:-2.2800 dloss:1.0268
Episode:169 meanR:11.5000 R:10.0 gloss:-2.3155 dloss:0.6442
Episode:170 meanR:11.4762 R:10.0 gloss:-2.3164 dloss:0.5485
Episode:171 meanR:11.4531 R:10.0 gloss:-2.3584 dloss:1.1065
Episode:172 meanR:11.4154 R:9.0 gloss:-2.4079 dloss:0.9967
Episode:173 meanR:11.3939 R:10.0 gloss:-2.4097 dloss:0.4786
Episode:174 meanR:11.3731 R:10.0 gloss:-2.3999 dloss:0.6714
Episode:175 meanR:11.3529 R:10.0 gloss:-2.4668 dloss:1.4253
Episode:176 meanR:11.3188 R:9.0 gloss:-2.4971 dloss:1.4587
Episode:177 meanR:11.3000 R:10.0 gloss:-2.5321 dloss:0.5312
Episode:178 meanR:11.2958 R:11.0 gloss:-2.5492 dloss:0.4013
Episode:179 meanR:11.2778 R:10.0 gloss:-2.5811 dloss:0.6086
Episode:180 meanR:11.2877 R:12.0 gloss:-2.6124 dloss:1.1243
Episode:181 meanR:11.2703 R:10.0 gloss:-2.6568 dloss:0.7817
Episode:182 meanR:11.2533 R:10.0 gloss:-2.6721 dloss:0.3712
Episode:183 meanR:11.2500 R:11.0 gloss:-2.6928 dloss:0.4314
Episode:184 meanR:11.2468 R:11.0 gloss:-2.

Episode:306 meanR:11.2600 R:11.0 gloss:-5.1206 dloss:0.8660
Episode:307 meanR:11.3200 R:15.0 gloss:-5.1646 dloss:0.8628
Episode:308 meanR:11.4100 R:18.0 gloss:-5.2081 dloss:0.8938
Episode:309 meanR:11.4900 R:18.0 gloss:-5.2804 dloss:0.8722
Episode:310 meanR:11.5000 R:11.0 gloss:-5.2912 dloss:0.9519
Episode:311 meanR:11.5400 R:14.0 gloss:-5.3388 dloss:0.7825
Episode:312 meanR:11.6000 R:16.0 gloss:-5.3764 dloss:0.7274
Episode:313 meanR:11.6400 R:14.0 gloss:-5.4257 dloss:0.7002
Episode:314 meanR:11.7100 R:15.0 gloss:-5.4690 dloss:0.7205
Episode:315 meanR:11.7400 R:12.0 gloss:-5.4728 dloss:0.7473
Episode:316 meanR:11.8100 R:17.0 gloss:-5.4955 dloss:0.7206
Episode:317 meanR:11.8500 R:14.0 gloss:-5.5438 dloss:0.8219
Episode:318 meanR:11.8800 R:17.0 gloss:-5.6031 dloss:0.8991
Episode:319 meanR:11.9100 R:13.0 gloss:-5.6021 dloss:0.8548
Episode:320 meanR:11.9800 R:17.0 gloss:-5.6026 dloss:0.7974
Episode:321 meanR:12.0400 R:17.0 gloss:-5.6259 dloss:0.9413
Episode:322 meanR:12.1100 R:16.0 gloss:-

Episode:443 meanR:13.5000 R:9.0 gloss:-7.2205 dloss:1.9793
Episode:444 meanR:13.4200 R:9.0 gloss:-7.2109 dloss:1.7426
Episode:445 meanR:13.3400 R:10.0 gloss:-7.2216 dloss:1.3430
Episode:446 meanR:13.2900 R:9.0 gloss:-7.2200 dloss:0.9951
Episode:447 meanR:13.2500 R:10.0 gloss:-7.2526 dloss:0.9635
Episode:448 meanR:13.1900 R:9.0 gloss:-7.2849 dloss:1.2811
Episode:449 meanR:13.1800 R:12.0 gloss:-7.2959 dloss:2.4381
Episode:450 meanR:13.1400 R:9.0 gloss:-7.3173 dloss:4.7240
Episode:451 meanR:13.2000 R:18.0 gloss:-7.4236 dloss:3.5783
Episode:452 meanR:13.2600 R:20.0 gloss:-7.5587 dloss:2.8205
Episode:453 meanR:13.2900 R:16.0 gloss:-7.4498 dloss:3.7387
Episode:454 meanR:13.3600 R:19.0 gloss:-7.2807 dloss:3.4825
Episode:455 meanR:13.2700 R:9.0 gloss:-7.1677 dloss:3.9715
Episode:456 meanR:13.2600 R:11.0 gloss:-7.1195 dloss:3.4549
Episode:457 meanR:13.2100 R:11.0 gloss:-7.1018 dloss:3.2221
Episode:458 meanR:13.1700 R:8.0 gloss:-7.0944 dloss:2.7728
Episode:459 meanR:13.0800 R:10.0 gloss:-7.0891 

Episode:580 meanR:16.5300 R:28.0 gloss:-7.8339 dloss:0.9372
Episode:581 meanR:16.6900 R:30.0 gloss:-7.9244 dloss:1.2555
Episode:582 meanR:16.7200 R:14.0 gloss:-8.0139 dloss:1.0236
Episode:583 meanR:16.6900 R:22.0 gloss:-8.0323 dloss:0.9181
Episode:584 meanR:16.7100 R:15.0 gloss:-8.0141 dloss:0.8527
Episode:585 meanR:16.7400 R:14.0 gloss:-8.0005 dloss:0.8050
Episode:586 meanR:16.7700 R:16.0 gloss:-8.0154 dloss:0.7738
Episode:587 meanR:16.7300 R:11.0 gloss:-8.0094 dloss:0.9191
Episode:588 meanR:16.7100 R:11.0 gloss:-7.9697 dloss:1.0431
Episode:589 meanR:16.8200 R:20.0 gloss:-7.9046 dloss:1.0869
Episode:590 meanR:16.8300 R:15.0 gloss:-7.8263 dloss:1.0655
Episode:591 meanR:16.9800 R:26.0 gloss:-7.8307 dloss:0.8010
Episode:592 meanR:16.9500 R:20.0 gloss:-7.8803 dloss:0.7669
Episode:593 meanR:16.9900 R:16.0 gloss:-7.9338 dloss:0.8905
Episode:594 meanR:17.0000 R:18.0 gloss:-7.9719 dloss:1.1799
Episode:595 meanR:17.0500 R:16.0 gloss:-8.0571 dloss:1.0460
Episode:596 meanR:17.0700 R:19.0 gloss:-

...

Episode:3999 meanR:14.2700 R:20.0 gloss:-72.8875 dloss:44.2860
Episode:4000 meanR:14.2600 R:17.0 gloss:-73.1919 dloss:41.7040
Episode:4001 meanR:14.2900 R:18.0 gloss:-73.4782 dloss:41.5895
Episode:4002 meanR:14.2900 R:15.0 gloss:-73.6841 dloss:41.3092
Episode:4003 meanR:14.3300 R:15.0 gloss:-73.8595 dloss:39.8720
Episode:4004 meanR:14.3800 R:17.0 gloss:-73.8455 dloss:37.0709
Episode:4005 meanR:14.4000 R:14.0 gloss:-73.8344 dloss:42.8537
Episode:4006 meanR:14.4400 R:21.0 gloss:-74.0306 dloss:48.0082
Episode:4007 meanR:14.4600 R:16.0 gloss:-74.4056 dloss:49.0992
Episode:4008 meanR:14.4600 R:17.0 gloss:-74.6004 dloss:37.5567
Episode:4009 meanR:14.5000 R:14.0 gloss:-74.6056 dloss:45.3051
Episode:4010 meanR:14.5600 R:17.0 gloss:-74.2714 dloss:41.4315
Episode:4011 meanR:14.6900 R:23.0 gloss:-74.0609 dloss:39.0253
Episode:4012 meanR:14.7600 R:20.0 gloss:-73.8772 dloss:32.0850
Episode:4013 meanR:14.9000 R:25.0 gloss:-74.0512 dloss:37.5125
Episode:4014 meanR:15.0400 R:24.0 gloss:-74.2037 dloss:

Episode:4130 meanR:13.4900 R:11.0 gloss:-71.0713 dloss:25.7714
Episode:4131 meanR:13.3800 R:10.0 gloss:-71.1917 dloss:28.2321
Episode:4132 meanR:13.2500 R:10.0 gloss:-71.2075 dloss:27.8264
Episode:4133 meanR:13.0800 R:12.0 gloss:-71.0849 dloss:26.2357
Episode:4134 meanR:12.9800 R:11.0 gloss:-71.3103 dloss:32.6079
Episode:4135 meanR:12.8700 R:12.0 gloss:-71.3586 dloss:29.2540
Episode:4136 meanR:12.8200 R:10.0 gloss:-71.3146 dloss:30.1146
Episode:4137 meanR:12.7500 R:11.0 gloss:-71.3702 dloss:28.9348
Episode:4138 meanR:12.6600 R:10.0 gloss:-71.3791 dloss:30.3297
Episode:4139 meanR:12.5900 R:10.0 gloss:-71.4062 dloss:27.6707
Episode:4140 meanR:12.5800 R:13.0 gloss:-71.3776 dloss:29.3608
Episode:4141 meanR:12.5800 R:9.0 gloss:-71.6110 dloss:29.7263
Episode:4142 meanR:12.5700 R:10.0 gloss:-71.7313 dloss:26.2434
Episode:4143 meanR:12.5600 R:9.0 gloss:-71.7628 dloss:25.1818
Episode:4144 meanR:12.5400 R:9.0 gloss:-71.7754 dloss:27.8197
Episode:4145 meanR:12.5200 R:10.0 gloss:-71.6964 dloss:24.

Episode:4261 meanR:11.4600 R:13.0 gloss:-69.2919 dloss:26.1880
Episode:4262 meanR:11.4300 R:10.0 gloss:-69.2059 dloss:29.5230
Episode:4263 meanR:11.4400 R:9.0 gloss:-69.3617 dloss:30.4310
Episode:4264 meanR:11.4400 R:12.0 gloss:-69.4158 dloss:33.3656
Episode:4265 meanR:11.4300 R:9.0 gloss:-69.3059 dloss:30.4137
Episode:4266 meanR:11.4300 R:9.0 gloss:-69.4571 dloss:26.8375
Episode:4267 meanR:11.4600 R:16.0 gloss:-69.3382 dloss:30.7320
Episode:4268 meanR:11.4800 R:10.0 gloss:-69.3957 dloss:24.5530
Episode:4269 meanR:11.4800 R:10.0 gloss:-69.5694 dloss:28.4567
Episode:4270 meanR:11.4400 R:11.0 gloss:-69.5210 dloss:33.1705
Episode:4271 meanR:11.4000 R:9.0 gloss:-69.4755 dloss:26.1113
Episode:4272 meanR:11.3600 R:9.0 gloss:-69.4827 dloss:26.9103
Episode:4273 meanR:11.3200 R:10.0 gloss:-69.3354 dloss:25.0228
Episode:4274 meanR:11.3900 R:16.0 gloss:-69.3783 dloss:28.9698
Episode:4275 meanR:11.4200 R:13.0 gloss:-69.4280 dloss:25.1792
Episode:4276 meanR:11.4400 R:11.0 gloss:-69.6559 dloss:23.47

Episode:4392 meanR:13.1800 R:13.0 gloss:-68.5709 dloss:88.6803
Episode:4393 meanR:13.1700 R:15.0 gloss:-68.2770 dloss:79.0692
Episode:4394 meanR:13.2100 R:20.0 gloss:-67.8333 dloss:66.3963
Episode:4395 meanR:13.2000 R:16.0 gloss:-67.5628 dloss:46.7898
Episode:4396 meanR:13.1200 R:8.0 gloss:-67.4963 dloss:42.1032
Episode:4397 meanR:13.0700 R:16.0 gloss:-67.2178 dloss:43.4668
Episode:4398 meanR:13.2200 R:26.0 gloss:-67.1168 dloss:46.7730
Episode:4399 meanR:13.4400 R:34.0 gloss:-67.1709 dloss:40.6664
Episode:4400 meanR:13.5300 R:23.0 gloss:-67.2879 dloss:52.1088
Episode:4401 meanR:13.4700 R:12.0 gloss:-67.3765 dloss:60.9088
Episode:4402 meanR:13.4500 R:9.0 gloss:-67.4810 dloss:46.0971
Episode:4403 meanR:13.4200 R:11.0 gloss:-67.5380 dloss:44.4195
Episode:4404 meanR:13.3600 R:11.0 gloss:-67.6773 dloss:46.9679
Episode:4405 meanR:13.2800 R:11.0 gloss:-67.7248 dloss:43.9014
Episode:4406 meanR:13.2000 R:9.0 gloss:-67.6801 dloss:42.3423
Episode:4407 meanR:13.2000 R:9.0 gloss:-67.7454 dloss:38.5

Episode:4523 meanR:11.7600 R:13.0 gloss:-64.8081 dloss:19.8311
Episode:4524 meanR:11.8100 R:15.0 gloss:-64.7529 dloss:24.3870
Episode:4525 meanR:11.8300 R:11.0 gloss:-64.6112 dloss:24.3375
Episode:4526 meanR:11.8300 R:13.0 gloss:-64.4620 dloss:24.3163
Episode:4527 meanR:11.7800 R:10.0 gloss:-64.5665 dloss:22.0020
Episode:4528 meanR:11.6900 R:11.0 gloss:-64.4845 dloss:28.1966
Episode:4529 meanR:11.6500 R:10.0 gloss:-64.8210 dloss:27.0880
Episode:4530 meanR:11.6700 R:13.0 gloss:-64.9149 dloss:25.2350
Episode:4531 meanR:11.7700 R:19.0 gloss:-65.0665 dloss:28.5576
Episode:4532 meanR:11.7800 R:12.0 gloss:-65.2319 dloss:31.0336
Episode:4533 meanR:11.6800 R:8.0 gloss:-65.1316 dloss:27.6534
Episode:4534 meanR:11.6500 R:8.0 gloss:-65.0383 dloss:24.0890
Episode:4535 meanR:11.6400 R:9.0 gloss:-65.0746 dloss:26.6515
Episode:4536 meanR:11.6700 R:12.0 gloss:-65.2350 dloss:29.7864
Episode:4537 meanR:11.6700 R:9.0 gloss:-65.1753 dloss:38.9890
Episode:4538 meanR:11.6700 R:11.0 gloss:-64.9338 dloss:33.4

In [ ]:
# np.mean(loss_batch, axis=0)

# Visualizing training

Below I'll plot the total rewards for each episode. I'm plotting the rolling average too, in blue.

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

def running_mean(x, N):
    cumsum = np.cumsum(np.insert(x, 0, 0)) 
    return (cumsum[N:] - cumsum[:-N]) / N 

In [ ]:
eps, arr = np.array(episode_rewards_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Total rewards')

In [ ]:
eps, arr = np.array(rewards_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Total rewards')

In [ ]:
eps, arr = np.array(loss_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Total rewards')

## Testing

Let's checkout how our trained agent plays the game.

In [35]:
# Creating a gym env
import gym

# env = gym.make('CartPole-v0')
env = gym.make('CartPole-v1')

# A training graph session
with tf.Session(graph=graph) as sess:
    #sess.run(tf.global_variables_initializer())
    #saver.restore(sess, 'checkpoints/model.ckpt')    
    saver.restore(sess, tf.train.latest_checkpoint('checkpoints'))
    
    # Episodes/epochs
    for _ in range(10):
        state = env.reset()
        total_reward = 0

        # Steps/batches
        while True:
            env.render()
            
            action, _ = act(sess, state)
            
            state, reward, done, _ = env.step(action)
            
            total_reward += reward
            
            if done:
                break
                
        # Closing the env
        print('total_reward: {}'.format(total_reward))

# Close the env at the end
env.close()

INFO:tensorflow:Restoring parameters from checkpoints/model.ckpt


/home/arasdar/anaconda3/lib/python3.6/site-packages/gym/envs/registration.py:14: PkgResourcesDeprecationWarning: Parameters to load are deprecated.  Call .resolve and .require separately.
  result = entry_point.load(False)


total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0


## Extending this

So, Cart-Pole is a pretty simple game. However, the same model can be used to train an agent to play something much more complicated like Pong or Space Invaders. Instead of a state like we're using here though, you'd want to use convolutional layers to get the state from the screen images.

![Deep Q-Learning Atari](assets/atari-network.png)

I'll leave it as a challenge for you to use deep Q-learning to train an agent to play Atari games. Here's the original paper which will get you started: http://www.davidqiu.com:8888/research/nature14236.pdf.